In [1]:
import pandas as pd
import numpy as np

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

# URL of the webpage
url = 'https://www.fantasypros.com/nfl/adp/ppr-overall.php'

# Fetch the HTML content from the URL
response = requests.get(url)
if response.status_code == 200:
    html_content = response.text

    # Create a BeautifulSoup object
    soup = BeautifulSoup(html_content, 'html.parser')

    # Find all player names
    player_names = [a.text.strip() for a in soup.find_all('a', attrs={'class': 'player-name'})]

    # Create a DataFrame from the player names
    FantasyPros = pd.DataFrame(player_names, columns=['Player Name'])

    # Add a column for the order, starting at 1
    FantasyPros['FantasyPros Rank'] = range(1, len(FantasyPros) + 1)

    # Print the DataFrame
    print(FantasyPros)
else:
    print(f"Failed to retrieve the webpage. Status code: {response.status_code}")


             Player Name  FantasyPros Rank
0    Christian McCaffrey                 1
1            CeeDee Lamb                 2
2            Tyreek Hill                 3
3          Ja'Marr Chase                 4
4       Justin Jefferson                 5
..                   ...               ...
404      Aaron Shampklin               405
405       Frank Gore Jr.               406
406        Antoine Green               407
407       Deneric Prince               408
408      Jerick McKinnon               409

[409 rows x 2 columns]


In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL of the webpage
url = 'https://www.footballguys.com/adp'

# Fetch the HTML content from the URL
response = requests.get(url)
if response.status_code == 200:
    html_content = response.text

    # Create a BeautifulSoup object
    soup = BeautifulSoup(html_content, 'html.parser')

    # List to hold player names and their ESPN values
    player_data = []
    rows = soup.find_all('tr')

    for row in rows:
        # Extract the player name
        name_td = row.find('td', class_='name sticky-col text-start')
        if name_td:
            name_a = name_td.find('a')
            if name_a:
                player_name = name_a.get_text().strip()
                # Extract the 9th <td> element (index 8) for ESPN value
                tds = row.find_all('td')
                if len(tds) >= 9:
                    espn_value = tds[8].get_text().strip()
                    # Append the player name and ESPN value as a tuple to the list
                    player_data.append((player_name, espn_value))

    # Create a DataFrame from the list
    ESPN = pd.DataFrame(player_data, columns=['Player Name', 'ESPN Rank'])

    # Convert ESPN Rank to numeric for proper sorting, handling non-numeric cases
    ESPN['ESPN Rank'] = pd.to_numeric(ESPN['ESPN Rank'], errors='coerce')
    ESPN = ESPN.dropna().sort_values('ESPN Rank')  # Drop rows where conversion failed and sort

    # Print the DataFrame
    print(ESPN)
else:
    print(f"Failed to retrieve the webpage. Status code: {response.status_code}")


               Player Name  ESPN Rank
0      Christian McCaffrey        1.0
1              CeeDee Lamb        2.0
2              Tyreek Hill        3.0
3            Ja'Marr Chase        4.0
4              Breece Hall        5.0
..                     ...        ...
473          Kyle Juszczyk      246.0
313  Donovan Peoples-Jones      247.0
223          D.J. Chark Jr      248.0
345   Tampa Bay Buccaneers      249.0
368         Denver Broncos      250.0

[250 rows x 2 columns]


In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL of the webpage
url = 'https://www.footballguys.com/adp'

# Fetch the HTML content from the URL
response = requests.get(url)
if response.status_code == 200:
    html_content = response.text

    # Create a BeautifulSoup object
    soup = BeautifulSoup(html_content, 'html.parser')

    # List to hold player names and their Underdog PPR values
    player_data = []
    rows = soup.find_all('tr')

    for row in rows:
        # Extract the player name
        name_td = row.find('td', class_='name sticky-col text-start')
        if name_td:
            name_a = name_td.find('a')
            if name_a:
                player_name = name_a.get_text().strip()
                # Extract the Underdog value; it's typically another <td> element, adjust the index as needed
                tds = row.find_all('td')
                if len(tds) > 14:  # Ensure there are enough <td> elements
                    underdog_value = tds[14].get_text().strip()  # Change to the correct index for Underdog
                    player_data.append((player_name, underdog_value))

    # Create a DataFrame from the list
    Underdog = pd.DataFrame(player_data, columns=['Player Name', 'Underdog Rank'])

    # Convert 'Underdog Rank' to numeric for sorting, coerce errors to NaN
    Underdog['Underdog Rank'] = pd.to_numeric(Underdog['Underdog Rank'], errors='coerce')
    Underdog = Underdog.dropna().sort_values('Underdog Rank')  # Drop NaNs and sort by Underdog Rank

    # Print the DataFrame
    print(Underdog)
else:
    print(f"Failed to retrieve the webpage. Status code: {response.status_code}")


                 Player Name  Underdog Rank
0        Christian McCaffrey            1.0
1                CeeDee Lamb            2.0
2                Tyreek Hill            3.0
3              Ja'Marr Chase            4.0
5           Justin Jefferson            5.0
..                       ...            ...
478  Marcus Rosemy-Jacksaint          346.0
477               Tip Reiman          347.0
472           Dillon Johnson          348.0
476            Foster Moreau          349.0
480           Isaiah Spiller          350.0

[350 rows x 2 columns]


In [5]:
import pandas as pd

# Sample data setup (replace with your actual DataFrame fetching code)
# dESPN = pd.DataFrame({'Player Name': ['Player A', 'Player B'], 'ESPN Rank': [1, 2]})
# df_fantasypros = pd.DataFrame({'Player Name': ['Player A', 'Player B'], 'FantasyPros Rank': [2, 1]})
# df_underdog = pd.DataFrame({'Player Name': ['Player A', 'Player B'], 'Underdog Rank': [1, 2]})

# Merge ESPN with FantasyPros on 'Player Name'
df_merged = pd.merge(ESPN, FantasyPros, on='Player Name', how='left')

# Merge the result with Underdog rankings on 'Player Name'
df_merged = pd.merge(df_merged, Underdog, on='Player Name', how='left')

df_merged.fillna('-', inplace=True)

# Check the resulting DataFrame
print(df_merged)


               Player Name  ESPN Rank FantasyPros Rank Underdog Rank
0      Christian McCaffrey        1.0              1.0           1.0
1              CeeDee Lamb        2.0              2.0           2.0
2              Tyreek Hill        3.0              3.0           3.0
3            Ja'Marr Chase        4.0              4.0           4.0
4              Breece Hall        5.0              8.0           8.0
..                     ...        ...              ...           ...
245          Kyle Juszczyk      246.0                -             -
246  Donovan Peoples-Jones      247.0            283.0         279.0
247          D.J. Chark Jr      248.0                -         212.0
248   Tampa Bay Buccaneers      249.0                -             -
249         Denver Broncos      250.0                -             -

[250 rows x 4 columns]


In [6]:
import requests
from bs4 import BeautifulSoup

# URL of the webpage
url = 'https://www.draftsharks.com/adp/half-ppr/yahoo/12'

# Fetch the HTML content from the URL
response = requests.get(url)
if response.status_code == 200:
    html_content = response.text

    # Create a BeautifulSoup object
    soup = BeautifulSoup(html_content, 'html.parser')

    # List to hold player names and data keys
    player_data = []

    # Find all tr elements with a 'data-key' attribute
    tr_elements = soup.find_all('tr', attrs={'data-key': True})

    print(f"Found {len(tr_elements)} tr elements with a 'data-key' attribute")

    for tr in tr_elements:
        # Navigate to the 'greaseBallCenter gb_skip' div to get the player name
        name_div = tr.select_one('.greaseBallCenter.gb_skip')
        if name_div:
            player_name = name_div.get_text().strip()
            data_key = tr['data-key']
            player_data.append((data_key, player_name))
        else:
            print(f"No player name found in tr element with data-key: {tr['data-key']}")

    # Print the player names and their data keys in order
    print("\nExtracted Player Data:")
    for data_key, player_name in player_data:
        print(f"data-key: {data_key}, Player: {player_name}")
else:
    print(f"Failed to retrieve the webpage. Status code: {response.status_code}")


Found 0 tr elements with a 'data-key' attribute

Extracted Player Data:
